In [25]:
from copy import deepcopy

In [34]:
"""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

REGS_TEST = {"A": 729, "B": 0, "C": 0}
PROG_TEST = [int(i) for i in "0,1,5,4,3,0".split(',')]

"""Register A: 123729
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""
REGS_TEST_1 = {"A": 123729, "B": 0, "C": 0}
PROG_TEST_1 = [int(i) for i in "0,3,5,4,3,0".split(',')]

"""Register A: 64012472
Register B: 0
Register C: 0

Program: 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0"""
REGS = {"A": 64012472, "B": 0, "C": 0}
PROG = [int(i) for i in "2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0".split(',')]

In [32]:
# There are two types of operands; each instruction specifies the type of its operand. 
# The value of a literal operand is the operand itself. 
# For example, the value of the literal operand 7 is the number 7. 
# The value of a combo operand can be found as follows:

# Combo operands 0 through 3 represent literal values 0 through 3.
# Combo operand 4 represents the value of register A.
# Combo operand 5 represents the value of register B.
# Combo operand 6 represents the value of register C.
# Combo operand 7 is reserved and will not appear in valid programs.
def combo(o, regs):
    if 0 <= o <= 3:
        return o
    elif o == 4:
        return regs['A']
    elif o == 5:
        return regs['B']
    elif o == 6:
        return regs['C']
    else:
        raise ValueError("invalid or reserved")

# The adv instruction (opcode 0) performs division. The numerator is the value in the A register. 
# The denominator is found by raising 2 to the power of the instruction's combo operand. 
# (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) 
# The result of the division operation is truncated to an integer and then written to the A register.
def adv(o, regs, pt, state):
    num, den = regs['A'], (1 << combo(o, regs))
    regs['A'] = num // den
    pt += 2
    return regs, pt, state

# The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the 
# instruction's literal operand, then stores the result in register B.
def bxl(o, regs, pt, state):
    regs['B'] = regs['B'] ^ o
    pt += 2
    return regs, pt, state

# The bst instruction (opcode 2) calculates the value of its combo operand modulo 8 
# (thereby keeping only its lowest 3 bits), then writes that value to the B register.
def bst(o, regs, pt, state):
    regs['B'] = combo(o, regs) % 8
    pt += 2
    return regs, pt, state

# The jnz instruction (opcode 3) does nothing if the A register is 0. However, if the A register 
# is not zero, it jumps by setting the instruction pointer to the value of its literal operand; 
# if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.
def jnz(o, regs, pt, state):
    if regs['A'] == 0:
        pt += 2
        return regs, pt, state
    else:
        pt = o
        return regs, pt, state

# The bxc instruction (opcode 4) calculates the bitwise XOR of register B and register C, 
# then stores the result in register B. (For legacy reasons, this instruction reads an operand but ignores it.)
def bxc(o, regs, pt, state):
    regs['B'] = regs['B'] ^ regs['C']
    pt += 2
    return regs, pt, state

# The out instruction (opcode 5) calculates the value of its combo operand modulo 8, 
# then outputs that value. (If a program outputs multiple values, they are separated by commas.)
def out(o, regs, pt, state):
    pt += 2
    # print(f"{combo(o, regs) % 8},")
    state += f"{combo(o, regs) % 8}"
    return regs, pt, state

# The bdv instruction (opcode 6) works exactly like the adv instruction except that the result is stored in the B register. 
# (The numerator is still read from the A register.)
def bdv(o, regs, pt, state):
    num, den = regs['A'], (1 << combo(o, regs))
    regs['B'] = num // den
    pt += 2
    return regs, pt, state

# The cdv instruction (opcode 7) works exactly like the adv instruction except that the result is stored in the C register. 
# (The numerator is still read from the A register.)
def cdv(o, regs, pt, state):
    num, den = regs['A'], (1 << combo(o, regs))
    regs['C'] = num // den
    pt += 2
    return regs, pt, state

INS = {0: adv, 1: bxl, 2: bst, 3: jnz, 4: bxc, 5: out, 6: bdv, 7: cdv}

Part 1

In [33]:
regs = deepcopy(REGS)
prog = PROG
state = ""
pt = 0

while pt < len(prog)-1:
    oc = prog[pt]
    o = prog[pt+1]
    regs, pt, state = INS[oc](o, regs, pt, state)

",".join(list(state))

'1,0,2,0,5,7,2,1,3'

Part 2

In [ ]:
prog = PROG
orig_state = "".join([str(p) for p in prog])

i = 0
is_orig = False
while i < 100_000_000:
    regs = {"A": i, "B": 0, "C": 0}
    state = ""
    pt = 0
    while pt < len(prog)-1:
        is_diff = False
        oc = prog[pt]
        o = prog[pt+1]
        regs, pt, state = INS[oc](o, regs, pt, state)
        for k in range(min(len(state), len(orig_state))):
            if state[k] != orig_state[k]:
                is_diff = True
                break
        if is_diff:
            break
    if state == orig_state:
        print("Halt with i ", i)
        break
    if i % 1_000_000 == 0:
        print(i)
    i += 1

In [ ]:
# 2,4,1,7,7,5,0,3,1,7,4,1,5,5,3,0
